# Repaso Prueba 2

## Examen 2019 - P.4

Suponga que existe una base de datos con dos tablas relacionadas, RATINGS y MOVIES. La primera contiene calificaciones históricas de películas hechas por distintos usuarios (vale decir, una misma película puede tener muchas notas). La segunda contiene información de identificación de cada película. Ambas tablas contienen un campo para identificar a cada película llamado MOVIE_ID.

Escriba código en SQL que le permita conocer:

**1. \[10 puntos\] El nombre de las 5 mejores películas estrenadas desde el año 2000 a la fecha. Para responder esta pregunta, suponga que la tabla MOVIES contiene la variable "año de estreno" (YEAR_AIR), y la variable "nombre de película" (NAME). La tabla RATINGS contiene la variable RATING. Además, suponga que "mejor" quiere decir aquella película con el rating promedio más alto.**

Tabla RATINGS con tres variables: MOVIE_ID, USER_ID y RATING. \
Tabla MOVIES tiene tres variables: MOVIE_ID, YEAR y NOMBRE.

In [ ]:
'''
SELECT M_ID, AVG(RATING)
FROM RATINGS
GROUP BY MOVIE_ID
'''

**Nota**: con querys más complejas suele ser común asignar nombre a las tablas...

In [ ]:
'''
SELECT R.M_ID, AVG(R.RATING)
FROM RATINGS R
GROUP BY MOVIE_ID
'''

# se entiende que a la tabla RATINGS se le asigna el pseudónimo R
# esto es útil cuando se trabajan con varias tablas con columnas similares

Sacamos el promedio, que es útil, pero necesitamos filtrar ahora las películas estrenadas del 2000 en adelante.

In [ ]:
'''
SELECT M_ID, AVG(RATING)
FROM RATINGS JOIN MOVIES USING(MOVIE_ID)
WHERE YEAR >= 200
GROUP BY MOVIE_ID
'''

# este join está temporalmente añadiendo a la tabla RATINGS la información de MOVIES
# dado que en RATINGS hay múltiples observaciones para MOVIE_ID, YEAR y NOMBRE se repetirá...

Faltaría buscar las 5 mejores películas por nombre...

In [ ]:
'''
SELECT M_ID, AVG(RATING) AS NOTA_PROM
FROM RATINGS JOIN MOVIES USING (MOVIE_ID)
WHERE YEAR >= 200
GROUP BY MOVIE_ID, NOMBRE
ORDER BY NOTA_PROM DESC
LIMIT 5
'''

# LIMIT 5 sirve para indicar que nos arroje solo las primeras 5 observaciones que cumplan las condiciones
# ORDER BY VA SIEMPRE DESPUÉS DE GROUP BY

Si quisiera solo el nombre podría hacer una consulta anidada... 

In [ ]:
'''
SELECT NOMBRE
FROM (
    SELECT M_ID, AVG(RATING) AS NOTA_PROM
    FROM RATINGS JOIN MOVIES USING (MOVIE_ID)
    WHERE YEAR >= 200
    GROUP BY MOVIE_ID, NOMBRE
    ORDER BY NOTA_PROM DESC
    LIMIT 5
)
'''

¿Qué pasaría si RATINGS tiene MOVIE_ID pero en MOVIES se llama ID? En vez de usar USING (MOVIE_ID) tenemos que decir ON...

In [ ]:
'''
SELECT M_ID, AVG(RATING) AS NOTA_PROM
FROM RATINGS R JOIN MOVIES M ON R.MOVIE_ID = M.ID
WHERE YEAR >= 200
GROUP BY MOVIE_ID, NOMBRE
ORDER BY NOTA_PROM DESC
LIMIT 5
'''

**2. \[10 puntos\] El número de películas por año, desde el año 2000 a la fecha, que hayan tenido más de 100 calificaciones.**

Un primer paso bien marcado es contar ratings por película, dígase, para cada película cuantas calificaciones tiene cada una.

In [ ]:
'''
SELECT MOVIE_ID, COUNT(*) AS NUM
FROM RATINGS
GROUP BY MOVIE_ID
'''

¿Cómo podría señalar aquellas películas con más de 100 calificaciones?

In [ ]:
# método artesanal: consulta anidada...

# función HAVING, que puede ejecutarse post GROUP BY

'''
SELECT MOVIE_ID, COUNT(*) AS NUM
FROM RATINGS
GROUP BY MOVIE_ID
HAVING NUM >= 100
'''

Faltaría señalar la fecha...

In [ ]:
# alternativa 1

'''
SELECT YEAR, COUNT(*)
FROM (
    SELECT MOVIE_ID, YEAR, COUNT(*) AS NUM
    FROM RATINGS JOIN MOVIES USING (MOVIE_ID)
    GROUP BY MOVIE_ID, YEAR
    HAVING NUM >= 100
)
WHERE YEAR >= 2000
GROUP BY YEAR
'''

# alternativa 2

'''
WITH BASE AS (
    SELECT MOVIE_ID, COUNT(*) AS NUM
    FROM RATINGS
    GROUP BY MOVIE_ID
)
SELECT YEAR, COUNT(*)
FROM (
    SELECT MOVIE_ID, YEAR
    FROM BASE JOIN MOVIES USING (MOVIE_ID)
    WHERE NUM >= 100
)
WHERE YEAR >= 2000
GROUP BY YEAR
'''

## Examen 2020

(...) Después de las elecciones, su candidato (A) perdió, y le pidieron a ud. analizar los datos de votaciones para tratar de entender que pasó. Afortunadamente, consiguieron la información en una base de datos estructurada (SQL), con la siguiente descripción...

**b. (5 puntos) Construya una query que le permita saber el núemero de votos obtenidos por cada candidato, por carrera.**

Podríamos partir sumando por mesa...

In [ ]:
'''
SELECT MESAS, SUM(VA), SUM(VB)
FROM VOTOS
GROUP BY MESA
'''

Si lo quisiera hacer por carrera...

In [ ]:
'''
SELECT CARR, SUM(VA), SUM(VB)
FROM VOTOS JOIN MESAS USING (MESA_ID)
GROUP BY CARR
'''

In [1]:
import pandas as pd
import sqlite3

# Crear datos de muestra para las tablas MOVIES y RATINGS
data_movies = {
    "MOVIE_ID": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "NAME": ["Película A", "Película B", "Película C", "Película D", "Película E", 
             "Película F", "Película G", "Película H", "Película I", "Película J"],
    "YEAR_AIR": [2001, 2003, 2002, 2005, 2004, 2000, 2006, 2007, 2008, 2009]
}

data_ratings = {
    "MOVIE_ID": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 
                 5, 5, 5, 6, 7, 7, 7, 8, 9, 10, 10, 10, 10, 10, 10],
    "RATING": [4, 5, 3, 4, 5, 5, 4, 3, 4, 5, 4, 4, 5, 5, 4, 3, 3, 2, 4, 5, 
               5, 4, 5, 3, 4, 4, 4, 5, 5, 3, 4, 3, 3, 2, 5]
}

# Crear DataFrames de Pandas
df_movies = pd.DataFrame(data_movies)
df_ratings = pd.DataFrame(data_ratings)

# Crear una conexión a una base de datos en memoria
conn = sqlite3.connect(':memory:')

# Copiar los DataFrames a la base de datos
df_movies.to_sql('MOVIES', conn, index=False, if_exists='replace')
df_ratings.to_sql('RATINGS', conn, index=False, if_exists='replace')

# Verificar las tablas creadas
df_movies.head(), df_ratings.head()

(   MOVIE_ID        NAME  YEAR_AIR
 0         1  Película A      2001
 1         2  Película B      2003
 2         3  Película C      2002
 3         4  Película D      2005
 4         5  Película E      2004,
    MOVIE_ID  RATING
 0         1       4
 1         1       5
 2         1       3
 3         1       4
 4         1       5)

In [3]:
query_best_movies = """
SELECT M.NAME, AVG(R.RATING) as AVG_RATING
FROM MOVIES M
JOIN RATINGS R ON M.MOVIE_ID = R.MOVIE_ID
WHERE M.YEAR_AIR >= 2000
GROUP BY M.NAME
ORDER BY AVG_RATING DESC
LIMIT 5;
"""

# Ejecutar la consulta para las mejores películas
best_movies = pd.read_sql_query(query_best_movies, conn)
best_movies

,NAME,AVG_RATING
0,Película I,5.000000
1,Película H,5.000000
2,Película E,4.666667
3,Película C,4.666667
4,Película A,4.200000


In [5]:
query_movies_per_year = """
SELECT M.YEAR_AIR, COUNT(M.MOVIE_ID) AS NUM_MOVIES
FROM MOVIES M
WHERE M.YEAR_AIR >= 2000
AND M.MOVIE_ID IN (
    SELECT R.MOVIE_ID
    FROM RATINGS R
    GROUP BY R.MOVIE_ID
    HAVING COUNT(R.RATING) > 3
)
GROUP BY M.YEAR_AIR
ORDER BY M.YEAR_AIR;
"""

# Ejecutar la consulta para películas por año
movies_per_year = pd.read_sql_query(query_movies_per_year, conn)
movies_per_year

,YEAR_AIR,NUM_MOVIES
0,2001,1
1,2005,1
2,2009,1


In [19]:
query_movies_per_year = """
SELECT M.YEAR_AIR, COUNT(M.MOVIE_ID) AS NUM_MOVIES
FROM MOVIES M
WHERE M.YEAR_AIR >= 2000
AND M.MOVIE_ID IN (
    SELECT R.MOVIE_ID
    FROM RATINGS R
    GROUP BY R.MOVIE_ID
    HAVING COUNT(R.RATING) >= 3
)
GROUP BY M.YEAR_AIR
ORDER BY M.YEAR_AIR;
"""

# Ejecutar la consulta para películas por año
movies_per_year = pd.read_sql_query(query_movies_per_year, conn)
movies_per_year

,YEAR_AIR,NUM_MOVIES
0,2001,1
1,2002,1
2,2004,1
3,2005,1
4,2006,1
5,2009,1


In [18]:
query_movies_per_year = """
SELECT YEAR_AIR, COUNT(*) AS MOVIE_N
FROM (
    SELECT M.MOVIE_ID, M.YEAR_AIR
    FROM MOVIES M
    JOIN RATINGS R ON M.MOVIE_ID = R.MOVIE_ID
    GROUP BY M.MOVIE_ID, M.YEAR_AIR
    HAVING COUNT(R.MOVIE_ID) >= 3
) AS SUBQUERY
WHERE YEAR_AIR >= 2000
GROUP BY YEAR_AIR;
"""

# Ejecutar la consulta para películas por año
movies_per_year = pd.read_sql_query(query_movies_per_year, conn)
movies_per_year

,YEAR_AIR,MOVIE_N
0,2001,1
1,2002,1
2,2004,1
3,2005,1
4,2006,1
5,2009,1
